In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
print(dense_layer)
#norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)
#norm_layer = Lambda(lambda  x: K.l2_normalize(x), name='norm_layer')(dense_layer)

# Final Model
model = Model(inputs=[myInput], outputs=dense_layer)


Tensor("dense_layer/BiasAdd:0", shape=(?, 128), dtype=float32)


## Contruct the model structure

## Load weights, Set weights and Save the model

In [3]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
    
    if model.get_layer(name) != None:
        model.get_layer(name).set_weights(weights_dict[name])
    elif model.get_layer(name) != None:
        model.get_layer(name).set_weights(weights_dict[name])

In [4]:
model.save('./model/nn4.small2.lrn.h5')

In [5]:
!open .

'open' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
def img_to_encoding(image_path, model):
    img = cv2.imread(image_path, 1)
    img = cv2.resize(img,(96,96))
    img = img[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    x_train = np.reshape(x_train,(1,96,96,3))
    #print(x_train.T.shape)
    
    embedding = model.predict_on_batch(x_train)
    return embedding

In [7]:
database = {}
keys= ['AK','FR','Noise','SGV','SK','SM','SP']
for p in keys:
    database[p]=[]
    for pic in os.listdir("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces_large\\train"+'\\'+p):
        database[p] = database[p] +[img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces_large\\train"
                                                        +'\\'+p+'\\'+pic, model)]
    #print(database)
        
'''
database["FR"] = img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces\\train\\FR.jpg", model)
database["SGV"] = img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces\\train\\SGV.jpg",model)
database["SK"] = img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces\\train\\SK.jpg", model)
database["SM"] = img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces\\train\\SM.jpg", model)
database["SP"] = img_to_encoding("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\faces\\train\\SP.jpg", model)
'''
print(len(database['AK']))
print(len(database['FR']))
print(len(database['Noise']))
print(len(database['SGV']))
print(len(database['SK']))
print(len(database['SM']))
print(len(database['SP']))




160
169
34
160
170
161
154


In [8]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 10000
    
    # Loop over the database dictionary's names and encodings.
    for (name, lis) in database.items():
        dist=100000
        dist_lis = []
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        for db_enc in lis:
            #if(np.linalg.norm(encoding - db_enc )<=1.5):
            dist_lis = dist_lis + [np.linalg.norm(encoding - db_enc )]

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        #print(len(dist_lis))
        
        if(len(dist_lis)!=0):
            dist = sum(dist_lis)/len(dist_lis)
        #print(dist)
        if dist < min_dist :
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    
    print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        

    return min_dist, identity

In [10]:
import os
import pandas

labels = pd.read_csv('C:\\Users\Sai Teja\Desktop\ELL888-CNN\\labels_2.csv')
labels = np.array(labels) 
print(labels[0,:])
res = []
for img in os.listdir("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\cropped\\test_vid_2"):
    print(img)
    s =  img.split('.')[0]
    s = int(s[1:])
    _,iden = who_is_it("C:\\Users\Sai Teja\Desktop\ELL888-CNN\\cropped\\test_vid_2"+"\\"+img, database, model)
    if(iden == 'AK' and labels[s-1][0]=='A' and labels[s-1][1]=='K'):
        res = res + [1]
    elif(iden == 'FR' and labels[s-1][0]=='F' and labels[s-1][1]=='R'):
        res = res + [1]
    elif(iden == 'Noise' and labels[s-1][0]=='j' and labels[s-1][1]=='k'):
        res = res + [1]
    elif(iden == 'SGV' and labels[s-1][0]=='S' and labels[s-1][1]=='G'):
        res = res + [1]
    elif(iden == 'SK' and labels[s-1][0]=='S' and labels[s-1][1]=='K'):
        res = res + [1]
    elif(iden == 'SM' and labels[s-1][0]=='S' and labels[s-1][1]=='M'):
        res = res + [1]
    elif(iden == 'SP' and labels[s-1][0]=='S' and labels[s-1][1]=='P'):
        res = res + [1]
    else:
        res = res + [0]
print(res)
print(len(res))
print(sum(res)/len(res))

['F' 'R']
_1.jpg
it's AK, the distance is 1.76400446445
_100.jpg
it's SP, the distance is 1.45464242163
_1000.jpg
it's FR, the distance is 2.69232681023
_1003.jpg
it's AK, the distance is 1.7632142216
_1009.jpg
it's Noise, the distance is 1.01054233488
_1010.jpg
it's AK, the distance is 1.13757670075
_1013.jpg
it's SP, the distance is 1.1038421226
_1016.jpg
it's AK, the distance is 1.22032005563
_1018.jpg
it's Noise, the distance is 1.15500986927
_1019.jpg
it's AK, the distance is 1.6225200288
_102.jpg
it's AK, the distance is 1.45442404673
_1020.jpg
it's AK, the distance is 1.31879643202
_1022.jpg
it's AK, the distance is 0.994191100821
_1023.jpg
it's AK, the distance is 1.92967875749
_1026.jpg
it's Noise, the distance is 1.61998066481
_1028.jpg
it's SP, the distance is 1.37027370775
_1029.jpg
it's SGV, the distance is 1.60355682112
_103.jpg
it's AK, the distance is 1.27106661163
_1032.jpg
it's Noise, the distance is 0.960615810226
_1033.jpg
it's AK, the distance is 1.47680694684
_103

it's SP, the distance is 1.11741079835
_1258.jpg
it's AK, the distance is 1.15734211355
_1259.jpg
it's AK, the distance is 1.05690865479
_126.jpg
it's FR, the distance is 2.58322616371
_1260.jpg
it's AK, the distance is 1.54880366921
_1265.jpg
it's SP, the distance is 1.13811070501
_1266.jpg
it's AK, the distance is 1.22950818464
_1267.jpg
it's AK, the distance is 1.31252341568
_1268.jpg
it's AK, the distance is 1.63865905702
_1269.jpg
it's SGV, the distance is 1.8035384886
_127.jpg
it's AK, the distance is 1.90892596766
_1271.jpg
it's Noise, the distance is 0.953941380276
_1272.jpg
it's AK, the distance is 1.21261422597
_1275.jpg
it's AK, the distance is 1.83588268608
_1276.jpg
it's Noise, the distance is 1.04040590104
_1277.jpg
it's AK, the distance is 1.72276276946
_1278.jpg
it's AK, the distance is 1.21743660383
_1279.jpg
it's SP, the distance is 1.32819910251
_1280.jpg
it's FR, the distance is 2.5652369677
_1284.jpg
it's AK, the distance is 1.48634908795
_1285.jpg
it's Noise, the 

it's AK, the distance is 0.939489075914
_1486.jpg
it's AK, the distance is 1.19005801119
_1487.jpg
it's SGV, the distance is 1.94613239691
_1488.jpg
it's AK, the distance is 1.13137364648
_1490.jpg
it's AK, the distance is 1.36711042263
_1491.jpg
it's FR, the distance is 2.73904747723
_1493.jpg
it's AK, the distance is 1.66536730155
_1494.jpg
it's AK, the distance is 1.52243582308
_1498.jpg
it's SP, the distance is 1.21764173175
_1499.jpg
it's AK, the distance is 1.26145586111
_15.jpg
it's Noise, the distance is 1.7491827537
_1500.jpg
it's SP, the distance is 1.22676185393
_1502.jpg
it's Noise, the distance is 1.06864931654
_1503.jpg
it's AK, the distance is 1.84442189485
_1504.jpg
it's AK, the distance is 1.40172256306
_1505.jpg
it's Noise, the distance is 1.06174822064
_1507.jpg
it's AK, the distance is 1.07465808503
_1508.jpg
it's SP, the distance is 1.28170530943
_1509.jpg
it's AK, the distance is 1.49287664741
_151.jpg
it's AK, the distance is 1.27144282013
_1512.jpg
it's AK, the 

it's AK, the distance is 1.87587364018
_1736.jpg
it's AK, the distance is 1.78589076772
_1737.jpg
it's SP, the distance is 1.63869441949
_1739.jpg
it's AK, the distance is 1.25523695946
_174.jpg
it's Noise, the distance is 0.982661627671
_1740.jpg
it's AK, the distance is 1.63173103333
_1741.jpg
it's SGV, the distance is 2.4278171353
_1742.jpg
it's AK, the distance is 1.87908520177
_1744.jpg
it's AK, the distance is 1.78887154832
_1746.jpg
it's AK, the distance is 1.62404615954
_1747.jpg
it's FR, the distance is 2.49511121328
_1748.jpg
it's AK, the distance is 1.2081405241
_175.jpg
it's Noise, the distance is 1.55584231896
_1751.jpg
it's AK, the distance is 2.12709530368
_1753.jpg
it's AK, the distance is 1.83788051009
_1754.jpg
it's SP, the distance is 1.60420001482
_1755.jpg
it's AK, the distance is 1.66060419828
_1757.jpg
it's AK, the distance is 1.42297782935
_1758.jpg
it's SGV, the distance is 1.76572265849
_1759.jpg
it's Noise, the distance is 1.55853056206
_1760.jpg
it's AK, the

it's AK, the distance is 1.50151427165
_1974.jpg
it's Noise, the distance is 1.0323649715
_1975.jpg
it's SP, the distance is 1.25240663662
_1978.jpg
it's SP, the distance is 1.48645097288
_1979.jpg
it's Noise, the distance is 1.37033286165
_198.jpg
it's AK, the distance is 1.03645743262
_1981.jpg
it's AK, the distance is 1.47429904565
_1982.jpg
it's SP, the distance is 1.16666188217
_1983.jpg
it's SP, the distance is 1.36291729707
_1985.jpg
it's AK, the distance is 1.03211850077
_1987.jpg
it's FR, the distance is 3.29000157858
_1989.jpg
it's SP, the distance is 1.12712639261
_199.jpg
it's AK, the distance is 2.0889852725
_1990.jpg
it's SP, the distance is 1.42881763252
_1992.jpg
it's AK, the distance is 1.45434272736
_1993.jpg
it's SGV, the distance is 1.54388305768
_1995.jpg
it's AK, the distance is 1.65966878161
_1996.jpg
it's Noise, the distance is 0.986666944097
_1999.jpg
it's AK, the distance is 1.54716161974
_2.jpg
it's AK, the distance is 1.24518373385
_200.jpg
it's AK, the dist

...

it's AK, the distance is 1.53466902077
_2923.jpg
it's AK, the distance is 1.93317557126
_2924.jpg
it's SP, the distance is 1.24292584054
_2925.jpg
it's AK, the distance is 1.55337135419
_2926.jpg
it's AK, the distance is 1.17901240885
_2928.jpg
it's AK, the distance is 1.15248948783
_2929.jpg
it's AK, the distance is 1.46231038496
_293.jpg
it's FR, the distance is 2.77466604837
_2930.jpg
it's FR, the distance is 2.65311135413
_2931.jpg
it's AK, the distance is 1.48424445055
_2932.jpg
it's AK, the distance is 1.14871372618
_2934.jpg
it's AK, the distance is 1.0289086502
_2936.jpg
it's AK, the distance is 1.10155369118
_2937.jpg
it's SGV, the distance is 1.86891565323
_294.jpg
it's Noise, the distance is 1.01353093631
_2940.jpg
it's FR, the distance is 2.31759263813
_2942.jpg
it's AK, the distance is 1.89588149339
_2943.jpg
it's AK, the distance is 1.33996909
_2944.jpg
it's Noise, the distance is 1.81340862372
_2946.jpg
it's SP, the distance is 1.33283250982
_2947.jpg
it's AK, the distan

it's AK, the distance is 1.44256222472
_3392.jpg
it's AK, the distance is 1.82326040119
_3394.jpg
it's SM, the distance is 2.25154553067
_3397.jpg
it's SGV, the distance is 1.91663959101
_3398.jpg
it's Noise, the distance is 1.32104328099
_3399.jpg
it's SGV, the distance is 1.52872394659
_34.jpg
it's Noise, the distance is 1.66997073328
_340.jpg
it's Noise, the distance is 1.30454831965
_3403.jpg
it's AK, the distance is 1.16316865869
_3404.jpg
it's SK, the distance is 2.08690144665
_3405.jpg
it's AK, the distance is 1.16359742731
_3407.jpg
it's SGV, the distance is 1.91294677854
_3408.jpg
it's AK, the distance is 2.16772061437
_3409.jpg
it's Noise, the distance is 1.1566537829
_3411.jpg
it's Noise, the distance is 1.74941786948
_3412.jpg
it's SP, the distance is 1.23852994109
_3413.jpg
it's Noise, the distance is 0.980625157847
_3414.jpg
it's SP, the distance is 1.23986570711
_3415.jpg
it's AK, the distance is 1.59515124857
_3418.jpg
it's SP, the distance is 1.08920755711
_3419.jpg
it

it's SP, the distance is 1.11970507866
_432.jpg
it's AK, the distance is 0.959540860355
_434.jpg
it's Noise, the distance is 1.13340105379
_436.jpg
it's AK, the distance is 2.1649214752
_438.jpg
it's SGV, the distance is 1.92641091645
_44.jpg
it's AK, the distance is 1.72571247518
_440.jpg
it's AK, the distance is 1.93678868935
_441.jpg
it's SGV, the distance is 2.19610281065
_443.jpg
it's AK, the distance is 1.76335776746
_445.jpg
it's SP, the distance is 1.35791217584
_446.jpg
it's AK, the distance is 1.48602650501
_447.jpg
it's SGV, the distance is 1.99985442534
_448.jpg
it's Noise, the distance is 0.97675691808
_451.jpg
it's AK, the distance is 1.21056310833
_452.jpg
it's Noise, the distance is 1.00645868042
_453.jpg
it's SP, the distance is 1.28032716064
_454.jpg
it's AK, the distance is 1.09707773328
_455.jpg
it's SP, the distance is 1.18664095154
_456.jpg
it's AK, the distance is 1.61411981732
_458.jpg
it's AK, the distance is 1.57720759511
_459.jpg
it's SP, the distance is 1.28

it's SP, the distance is 1.33288824094
_678.jpg
it's Noise, the distance is 1.65064503515
_679.jpg
it's AK, the distance is 1.47720913365
_682.jpg
it's Noise, the distance is 1.62353320683
_683.jpg
it's AK, the distance is 2.31654715016
_685.jpg
it's AK, the distance is 1.88340658359
_686.jpg
it's AK, the distance is 1.0589265395
_687.jpg
it's AK, the distance is 1.37837736644
_688.jpg
it's SP, the distance is 1.40806586518
_692.jpg
it's Noise, the distance is 1.02457006363
_697.jpg
it's SP, the distance is 1.29359694541
_699.jpg
it's Noise, the distance is 0.998938592041
_7.jpg
it's Noise, the distance is 0.972300136791
_70.jpg
it's Noise, the distance is 1.03502434667
_701.jpg
it's AK, the distance is 1.11790223755
_702.jpg
it's AK, the distance is 1.2152807463
_703.jpg
it's SP, the distance is 1.38045035554
_704.jpg
it's AK, the distance is 1.1777039174
_705.jpg
it's Noise, the distance is 2.12162867364
_707.jpg
it's SGV, the distance is 1.68298821822
_709.jpg
it's AK, the distance 

it's SGV, the distance is 1.87724792659
_946.jpg
it's SP, the distance is 1.2878492312
_949.jpg
it's AK, the distance is 1.53916922398
_952.jpg
it's SGV, the distance is 1.6675478667
_953.jpg
it's FR, the distance is 2.33171441358
_958.jpg
it's SP, the distance is 1.15839477716
_959.jpg
it's SGV, the distance is 1.98014615253
_960.jpg
it's Noise, the distance is 1.6103592252
_961.jpg
it's AK, the distance is 2.10749181062
_962.jpg
it's AK, the distance is 1.28865990005
_963.jpg
it's SP, the distance is 1.2496371246
_964.jpg
it's AK, the distance is 1.07419351786
_965.jpg
it's SP, the distance is 1.45505981709
_966.jpg
it's SP, the distance is 1.27695182585
_968.jpg
it's AK, the distance is 1.05066302754
_97.jpg
it's AK, the distance is 1.10528690629
_970.jpg
it's Noise, the distance is 1.05192040696
_971.jpg
it's Noise, the distance is 0.961811184883
_972.jpg
it's AK, the distance is 1.29291302487
_974.jpg
it's AK, the distance is 1.52819768451
_976.jpg
it's AK, the distance is 1.34797